<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/roberta_large_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 6.6MB/s 
     |████████████████████████████████| 890kB 41.6MB/s 
     |████████████████████████████████| 3.2MB 46.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=db2a33af7829e36a5dd3beda5ab54b6bcd111fd77be31638866dbdd900e6cb75
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:42:50--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:42:50 (73.6 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('roberta-large')
model = AutoModelForMaskedLM.from_pretrained('roberta-large')

sequence = 'I live in {} and I speak {}.'

rows = []
for name in pd.read_csv('countries.csv')['en'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

In [4]:
pd.DataFrame(rows, columns=['country', 'language(top1)', 'language(top2)', 'language(top3)'])

,country,language(top1),language(top2),language(top3)
0,Afghanistan,English (59%),Arabic (14%),Persian (4%)
1,Albania,English (50%),Italian (10%),French (9%)
2,Algeria,French (74%),Arabic (12%),English (10%)
3,Andorra,French (25%),Spanish (22%),English (20%)
4,Angola,Portuguese (63%),English (18%),Spanish (8%)
5,Antigua and Barbuda,English (62%),Spanish (19%),Portuguese (6%)
6,Argentina,Spanish (46%),English (34%),French (8%)
7,Armenia,Armenian (34%),Russian (29%),English (21%)
8,Australia,English (73%),French (5%),Mandarin (4%)
9,Austria,German (59%),English (15%),French (7%)
